# Overview of Week 3

This week's assignment consists of two parts. The first part gives you an introduction to unsupervised learning. In particular, we focus on techniques for clustering and dimensionality reduction and how they can be applied to ecommerce data. As you work through the three clustering case studies, you will find yourself generating many intermediate datasets, trying different models, and tuning each model as you go along. There's a lot to keep track of.   

This is where Part 2 comes in. It is in Part 2 that we introduce ideas of **workflow management** and **computational reproducibility**. Workflow management means organising your project directory to manage your analysis' artefacts (visualisations, processed datasets, notebooks and utility functions and experiment results). Ideally, your code for these should be clearly commented with well chosen names. Computational reproducibility means someone else (including future you!) being able to take just the code and data, and reproduce your project, from its results and models to visualisations etc. How one decides to practice workflow management and computational reproducibility can be quite a personal decision. Therefore, we provide guidelines, not rules. The most important is having a system rather than no system at all. 

**note about the week**   
While week 1's assignment was guided, with specific instructions about what code to run, as we move on the assignments will involve less hand-holding. For this week, we include some instructions, but leave the specific implementations up to you. There are also many techniques we cover. Again, while we share some resources, we leave the bulk of the research and background reading up to you to manage for yourself. As a record of how well you have synthesised your readings, this notebook has space for writing a few paragraphs of explanations. We hope you won't be content with a shallow explanation, and that you use this space to apply the [Feynmann technique](https://collegeinfogeek.com/feynman-technique/) as a check of understanding. 

To tie Part 1 and Part 2 of the week together, we are not just asking for one Jupyter Notebook as a final deliverable. Instead, there are four deliverables to the week: 

**1. Part 1 Notebook I and II exercises presented at the end of the week job-interview style**

**2. A script that automates the data cleaning and feature engineering steps you take so your findings are reproducible (see Part 2 Notebook I for more info)**

**3. A repo README.md (see Part 2 Notebook I for more info)**

**4  A record of software package versions and software environment used (see Part 2 Notebook I for more info)**   

**recap of the objectives for the first 6 weeks:**  
We aim to broadly cover a wide range of Machine Learning algorithms so that you can: 
- handle the technical demands of a 100E given some guidance on the right direction to take 
- can handle a technical job interview and get hired 

*materials for unsupervised learning adapted from William Thji* 

# Part I : Unsupervised Learning 
Unsupervised Learning refers to a set of machine learning techniques where no output variables (Y) are given. Only the input variables (X) are available and our job is to find patterns in X. You may read more about it from *pg 485 from Hastie and Tibshirani's Elements of Statistical Learning* available [here](https://web.stanford.edu/~hastie/Papers/ESLII.pdf). 

ESL by Hastie et. al with be the primary reference for this week, although feel free to source for your own books and links. 

## Short introduction to clustering 
Clustering puts datapoints into subsets so that datapoints within a cluster are more closely related to one another compared to datapoints in another cluster. More information is available from page 501 of *Elements of Statistical Learning*. 

Some quick points: 
- Clustering is extremely useful to many fields: 
    - Customer segmentation for personalised product recommendations
    - Topic identification to relieve the need to manually vet documents 
    - Image or geo-spatial segmentation to optimised supply and demand (Gojek does this) 
    - and maybe most importantly, getting a sense of the data before starting to model it. 

- Some examples of clustering algorithms: 
    - KMeans
    - Gaussian Mixture Models for drawing soft clustering boundaries instead of hard ones 
    - Hierarchical clustering
    - DBScan for density-based clustering for anomaly detection 
    - Co-clustering
    - Biclustering for analysing genes

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Deliverable 1: Complete the Clustering Case Study: Using PCA and clustering to uncover customer segments 
Context: The dataset we will be working with contains ecommerce transactions from a UK-based online retails store. The dataset is available on [Kaggle](https://www.kaggle.com/carrie1/ecommerce-data/home) or the UCI Machine Learning Repository. The dataset is quite small, so we have also included it inside the `data` folder inside this repo as `data/raw/data.csv`. 

From the Kaggle website: 

"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."

### Case Study Goal 
Cluster the dataset into meaningful customer segments. This means creating clusters that are statistically robust and that make business sense. 


In [29]:
import pandas as pd 
df = pd.read_csv('data/raw/data.csv', encoding='ISO-8859-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### Cleaning data 

Some data types are muddled, there are duplicates, NA values and unreasonable values hiding in the columns 

1. Clean the dataset. You may want to list the steps taken to clean the data and encapsulate the steps inside their own functions so they can be reused. Organise the functions into their own library

In [2]:
def isDataClean(df):
    print("Numeric Data")
    print(df.describe())
    print("")
    print("Categorical Data")
    print(df.describe(include=['O']))
    print("")
    print("NAs in Data")
    print(df.isnull().any())
    print("\nSize of df")
    print(df.shape)

In [39]:
isDataClean(df)

Numeric Data
            Quantity      UnitPrice     CustomerID
count  397189.000000  397189.000000  397189.000000
mean       11.316867       3.119739   15293.361866
std        22.569099      22.117001    1712.710929
min         1.000000       0.000000   12347.000000
25%         2.000000       1.250000   13969.000000
50%         5.000000       1.950000   15157.000000
75%        12.000000       3.750000   16794.000000
max       360.000000    8142.750000   18287.000000

Categorical Data
       InvoiceNo StockCode                         Description  \
count     397189    397189                              397189   
unique     18392      3663                                3875   
top       576339    85123A  WHITE HANGING HEART T-LIGHT HOLDER   
freq         542      2026                                2019   

             InvoiceDate         Country  
count             397189          397189  
unique             17163              37  
top     11/14/2011 15:27  United Kingdom  
freq   

In [30]:
df=df[df['CustomerID'].notnull()]
# isDataClean(df)

In [31]:
df=df[df['Quantity']>0]
# isDataClean(df)

In [32]:
Quantity_min,Quantity_max=df['Quantity'].mean()-2* np.std(df['Quantity']),df['Quantity'].mean()+2* np.std(df['Quantity'])
df=df[(df['Quantity']>=Quantity_min) & (df['Quantity']<=Quantity_max)]
# isDataClean(df)

In [ ]:
# x=df[['CustomerID','InvoiceNo']].drop_duplicates().groupby(by=['InvoiceNo'])['CustomerID'].nunique()
# x_df = x.to_frame().reset_index()
# x_df.sort_values('CustomerID',ascending=False)

### EDA

In [21]:
df['Country'].value_counts(normalize=True)

United Kingdom          0.890556
Germany                 0.022760
France                  0.020998
EIRE                    0.018198
Spain                   0.006256
Netherlands             0.005869
Belgium                 0.005113
Switzerland             0.004638
Portugal                0.003681
Australia               0.002926
Norway                  0.002699
Italy                   0.001908
Channel Islands         0.001881
Finland                 0.001725
Cyprus                  0.001546
Sweden                  0.001083
Austria                 0.001002
Denmark                 0.000957
Poland                  0.000831
Japan                   0.000765
Israel                  0.000624
Unspecified             0.000614
Singapore               0.000559
Iceland                 0.000458
USA                     0.000451
Canada                  0.000378
Greece                  0.000365
Malta                   0.000282
United Arab Emirates    0.000171
European Community      0.000151
RSA       

In [34]:
df['CustomerID']=df['CustomerID'].astype(int).astype(int)

### Feature Engineering iteration #1 

Inside the dataset, each row contains information about an ecommerce transation. However, we want to cluster the data by customers, which means each row should instead contain information about a customer. 

1. Reshape the data to follow the format below: 
![alt text](customer.png)

The dataframe should have these columns: `['NoOfInvoices', 'NoOfUniqueItems', 'QuantityPerInvoice', 'TotalQuantity', 'UniqueItemsPerInvoice','UnitPriceMean','UnitPriceStd']` 
2. Save this dataset as an intermediate dataset 

In [40]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [52]:
# df.groupby('CustomerID')['CustomerID'].agg(['min', 'max'])

# df.groupby('CustomerID').agg({'InvoiceNo': 'nunique', 
#                               'StockCode': 'nunique',
#                               'Quantity':'avg',
#                               'Quantity':'sum',
#                               'StockCode' :'nunique',
#                               'UnitPrice':'mean',
#                               'UnitPrice':'std'
#                              })
(df.groupby('CustomerID')
    .agg({'InvoiceNo': 'nunique',
          'StockCode': 'nunique',
          'Quantity':'sum',
          'UnitPrice':'mean'
         })
    .rename(columns={'InvoiceNo': 'NoOfInvoices', 
                     'StockCode': 'NoOfUniqueItems',
                     'Quantity':'TotalQuantity',
                     'UnitPrice':'UnitPriceMean'})
)



,NoOfInvoices,UniqueItermsPerInvoice,TOtalQuantity,UnitPriceMean
CustomerID,,,,
12347,7,103,2458,2.644011
12348,4,22,2341,5.764839
12349,1,73,631,8.289041
12350,1,17,197,3.841176
12352,8,59,536,15.930706
12353,1,4,20,6.075000
12354,1,58,530,4.503793
12355,1,13,240,4.203846
12356,3,53,1591,3.201186


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397189 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      397189 non-null object
StockCode      397189 non-null object
Description    397189 non-null object
Quantity       397189 non-null int64
InvoiceDate    397189 non-null object
UnitPrice      397189 non-null float64
CustomerID     397189 non-null float64
Country        397189 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 37.3+ MB


### Hierarchical clustering iteration #1 [reference pg 520](https://web.stanford.edu/~hastie/Papers/ESLII.pdf)

1. Implement any more preprocessing or visualisation steps you feel are necessary to understand how you might build meaningful clusters from the data. 
2. Apply hierarchical clustering to the dataset. 
2. Experiment with different linkage algorithms. Visualise the resulting trees for average linkage, complete linkage and single linkage side-by-side. Which linkage algorithm works best? 
3. List two ways to improve the clustering and implement at least one. Track the results of the first iteration and second iteration (whether with Excel, TextEdit or within this notebook itself). 
4. Describe the clusters you selected and evaluate whether or not they form meaningful segments. 

In [ ]:
# your code here 

### K-means and GMM Clustering iteration #1 [reference pg 509](https://web.stanford.edu/~hastie/Papers/ESLII.pdf)
Apart from hierarchical clustering, we can also apply KMeans and Gaussian Mixture Models (GMM) on the data

1. Implement K-means clustering on the data, experimenting with different values of k 
2. Implement a Gaussian Mixture Model on the data, experimenting with different values for the number of components.  
3. Visualise how well K-means and GMMs succeed at separating subgroups in the data
4. List two ways to improve the clustering and implement at least one. 
5. Describe the clusters and evaluate whether or not they form meaningful segments. 

In [ ]:
# your code here 

### Outliers 

1. Do outliers influence the results of your Hierarchical Clustering, K-means and GMM models?  
2. Do the outliers themselves form clusters? 

In [ ]:
# your code here. 

### Validating clusters  

1. Use at least three techniques to validate that your chosen clusters correspond to meaningful customer segments. You may consider a combination of using visualisations and/or quantitative metrics (refer to Elements of Statistical Learning for some examples)

### Model selection

1. Explain when it is appropriate to use K-Means, GMM and Hierarchical Clustering 
2. Which model would you choose for this dataset? 

### Opening the black box of clustering [reference pg 503](https://web.stanford.edu/~hastie/Papers/ESLII.pdf)

1. In your own words, explain the broad idea behind building a proximity matrix to cluster data. (This should also be verbally presented during Monday's presentation). 
*Sometimes, typing values into MS Excel helps with building intuition

2. In your own words, how would you calculate the disimilarity between objects in a dataset? (This should also be verbally presented during Monday's presentation)

### PCA for Dimensionality Reduction 

1. Using the dataset that you have pre-processed, construct a pairplot again. How is it different from the a plot of the raw data and what is your interpretation?  
2. Apply PCA pre-processed dataset. More information on PCA here [link]
3. Create a plot of cumulative explained variance and number of components. How does this inform you about the best number of components to select? 
4. Create a plot of PC0 against PC1, coloured by the GMM's predictions on the normalised dataset with outliers removed for n_components =7
5. How would you connect the Princip